In [1]:
from pathlib import Path
import numpy as np
import os

In [8]:
class LearnVocabulary:
    def __init__(self, path2txt: str, alias: str) -> None:
        """
        I have prepared a program for you thanks to which you can quickly learn vocabulary. 
        You have several useful functions at your disposal: 
        1. .present_dictionary() - display the entire dictionary 
        2. .start_learning(num_of_word: int, num_of_letter_in_word: int) - learning specific words from the dictionary 
        3. .start_testing(num_of_word: int) - checks how you have learned specific words in the dictionary 

        For a function to work well, it needs 2 parameters: 
        :param: path2txt: str -- path to .txt file with your dictionary
        :param: alies: str -- set your alies used for file names  
        """

        self.path2txt = path2txt
        self.name = alias
        self.load_txt()
        self.preprocessing_name_of_language()
        self.preprocessing_text()
    
    def load_txt(self) -> None:
        with open(self.path2txt, encoding='utf-8') as f:
            self.list_of_sentence = f.readlines()
            self.name_of_language = self.list_of_sentence[0]
            self.list_of_sentence = self.list_of_sentence[1:]

    def preprocessing_text(self) -> None:
        list_of_word1 = []
        list_of_word2 = []
        for sentence in self.list_of_sentence:
            split_sentence = sentence.split('-')
            list_of_word1.append(split_sentence[0].strip())
            list_of_word2.append(split_sentence[-1].strip())
        self.language1 = {word1: word2 for word1, word2, in zip(list_of_word1, list_of_word2)}
        self.language2 = {word2: word1 for word1, word2, in zip(list_of_word1, list_of_word2)}

    def preprocessing_name_of_language(self) -> None:
        self.language = self.name_of_language.split('-')
        self.language = [lang.strip() for lang in self.language]


    def present_dictionary(self, option: int=1) -> None:
        '''
        Presentation of all words in the tested dictionary.
        :param: option: int=1 --> which language should be in the first column? (1- first, 2-second, another-error)
        '''
        print('##### All words in dictionary #####')
        if(option==1):
            idx = 1
            print(f'#### {self.language[0]} -- {self.language[1]}')
            for key, item in zip(self.language1.keys(), self.language1.values()):
                print(f'## {idx}. {key} -- {item}')
                idx += 1
        elif(option==2):
            idx = 1
            print(f'#### {self.language[1]} -- {self.language[0]}')
            for key, item in zip(self.language1.keys(), self.language1.values()):
                print(f'## {idx}. {key} -- {item}')
                idx += 1
        else:
            print('Error: option must be {1,2}')
    
    def create_learning_list(self) -> None:
        basic_path = self.path2txt[:-4]
        self.path2lang1 = basic_path + f'_ProgressList_{self.language[0]}_{self.name}.txt'
        self.path2lang2 = basic_path + f'_ProgressList_{self.language[1]}_{self.name}.txt'
        if Path(self.path2lang1).is_file():
            print(f' -- Your vocabulary list ({self.language[0]}) does exist!')
        else:
            print(f" -- Your vocabulary list ({self.language[0]}) doesn't exist! We create it")
            with open(self.path2lang1, 'w', encoding='utf-8') as f:
                for item in self.language1.keys():
                    f.write("%s\n" % item)
        if Path(self.path2lang2).is_file():
            print(f' -- Your vocabulary list ({self.language[1]}) does exist!')
        else:
            print(f" -- Your vocabulary list ({self.language[1]}) doesn't exist! We create it")
            with open(self.path2lang2, 'w', encoding='utf-8') as f:
                for item in self.language2.keys():
                    f.write("%s\n" % item)

    def load_your_vocabulary_list(self) -> None:
        with open(self.path2lang1, encoding='utf-8') as f:
            self.vocabulary_lang1_list = f.readlines()
            self.vocabulary_lang1_list = [word.strip().replace('\n', '') for word in self.vocabulary_lang1_list]
        with open(self.path2lang2, encoding='utf-8') as f:
            self.vocabulary_lang2_list = f.readlines()
            self.vocabulary_lang2_list = [word.strip().replace('\n', '') for word in self.vocabulary_lang2_list]

    def update_progress_list(self, language_task: int=1):
        if(language_task==1):
            os.remove(self.path2lang1)
            with open(self.path2lang1, 'w', encoding='utf-8') as f:
                for item in self.vocabulary_lang1_list:
                    f.write("%s\n" % item)
        elif(language_task==2):
            os.remove(self.path2lang2)
            with open(self.path2lang2, 'w', encoding='utf-8') as f:
                for item in self.vocabulary_lang2_list:
                    f.write("%s\n" % item)

    def rules_learn(self) -> None:
        print("""!PRINCIPLES OF LEARNING!
        You will get hints for each word in the form of additional letters.
        And then you check if you remember the word well. """)
   

    def rules_test(self) -> None:
        print("""!RULES TEST!
        You will get n words to translate in both languages. 
        Your progress list will be saved after each task. 
        If you translate the word correctly, it will be removed word from your progress list. 
        If you translate a word incorrectly, the word will be added to your progress list.""")

    def check_answer(self, word, input_word, language_task: int=1) -> bool:
        if (language_task==1):
            if(self.language1[word] == input_word):
                return True
            else:
                return False
        else: 
            if(self.language2[word] == input_word):
                return True
            else:
                return False

    def remove_word_from_progress_list(self, word: str, vocabulary_list: list, language_task: int=1) -> list:
        idx = vocabulary_list.index(word)
        vocabulary_list.remove(vocabulary_list[idx])
        return vocabulary_list

    def add_word_to_progress_list(self, word: str, vocabulary_list: list, language_task: int=1) -> list:
        idx = vocabulary_list.index(word)
        vocabulary_list.append(vocabulary_list[idx])
        return vocabulary_list

    def prompt(self, word: str, num_of_letter_in_word: int, language_task: int=1) -> str:
        if (language_task==1):
            correct_translation_word = self.language1[word]
        else: 
            correct_translation_word = self.language2[word]
        len_correct_translation_word = len(correct_translation_word)
        correct_letter_idx = np.random.choice(len(correct_translation_word), num_of_letter_in_word, replace=False)
        return " ".join([correct_translation_word[idx] if idx in correct_letter_idx else "_" for idx in range(len_correct_translation_word)])
    
    def translation_learn_task(self, num_of_word: int, num_of_letter_in_word: int=3, language_task: int=1) -> int:
        num_of_correct_answer = 0
        if(language_task==1):
            print(f"Exercise 1: Translate words from {self.language[0]} into {self.language[1]}.")
            if(len(self.vocabulary_lang1_list)==0):
                print("You already know everything!")
                return 100
            elif(len(self.vocabulary_lang1_list) < num_of_word):
                num_of_word = len(self.vocabulary_lang1_list)
            elif(num_of_word < 0):
                print(f"Seriously? How can there be {num_of_word} tasks on the test?")
                return -100
            list_of_word = np.random.choice(self.vocabulary_lang1_list, num_of_word, replace=False)
            # print(self.vocabulary_lang1_list)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} -->  {self.prompt(word, num_of_letter_in_word, language_task=1)}")
                input_word = str(input(f"# From {self.language[0]} into {self.language[1]} --{idx}. {word} --> {self.prompt(word, num_of_letter_in_word, language_task=1)}")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language1[word]}')")
            self.update_progress_list(language_task) 
        else:
            print(f"Exercise 2: Translate words from {self.language[1]} into {self.language[0]}.")
            if(len(self.vocabulary_lang2_list)==0):
                print("You already know everything!")
                return 100
            elif(len(self.vocabulary_lang2_list) <= num_of_word):
                num_of_word = len(self.vocabulary_lang2_list)
            elif(num_of_word < 0):
                print(f"Seriously? How can there be {num_of_word} tasks on the test?")
                return -100
            list_of_word = np.random.choice(self.vocabulary_lang2_list, num_of_word, replace=False)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} --> {self.prompt(word, num_of_letter_in_word, language_task=2)}")
                input_word = str(input(f"# From {self.language[1]} into {self.language[0]} -- {idx}. {word} --> {self.prompt(word, num_of_letter_in_word, language_task=2)}")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language2[word]}')")
            self.update_progress_list(language_task) 
        print(f'########## Your score: {num_of_correct_answer}/{num_of_word} ({100*num_of_correct_answer/num_of_word}%)')
        return int(100*num_of_correct_answer/num_of_word)
    
    def translation_test_task(self, num_of_word: int, language_task: int=1) -> int:
        num_of_correct_answer = 0
        if(language_task==1):
            print(f"Exercise 1: Translate words from {self.language[0]} into {self.language[1]}.")
            if(len(self.vocabulary_lang1_list)==0):
                print("You already know everything!")
                return 100
            elif(len(self.vocabulary_lang1_list) < num_of_word):
                num_of_word = len(self.vocabulary_lang1_list)
            elif(num_of_word < 0):
                print(f"Seriously? How can there be {num_of_word} tasks on the test?")
                return -100
            list_of_word = np.random.choice(self.vocabulary_lang1_list, num_of_word, replace=False)
            # print(self.vocabulary_lang1_list)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} --> ")
                input_word = str(input(f"# From {self.language[0]} into {self.language[1]} --{idx}. {word} --> ")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    self.vocabulary_lang1_list = self.remove_word_from_progress_list(word, self.vocabulary_lang1_list, language_task)
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language1[word]}')")
                    self.vocabulary_lang1_list = self.add_word_to_progress_list(word, self.vocabulary_lang1_list, language_task)
            self.update_progress_list(language_task) 
        else:
            print(f"Exercise 2: Translate words from {self.language[1]} into {self.language[0]}.")
            if(len(self.vocabulary_lang2_list)==0):
                print("You already know everything!")
                return 100
            elif(len(self.vocabulary_lang2_list) <= num_of_word):
                num_of_word = len(self.vocabulary_lang2_list)
            elif(num_of_word < 0):
                print(f"Seriously? How can there be {num_of_word} tasks on the test?")
                return -100
            list_of_word = np.random.choice(self.vocabulary_lang2_list, num_of_word, replace=False)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} --> ")
                input_word = str(input(f"# From {self.language[1]} into {self.language[0]} -- {idx}. {word} --> ")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    self.vocabulary_lang2_list = self.remove_word_from_progress_list(word, self.vocabulary_lang2_list, language_task)
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language2[word]}')")
                    self.vocabulary_lang2_list = self.add_word_to_progress_list(word, self.vocabulary_lang2_list, language_task)
            self.update_progress_list(language_task) 
        print(f'########## Your score: {num_of_correct_answer}/{num_of_word} ({100*num_of_correct_answer/num_of_word}%)')
        return int(100*num_of_correct_answer/num_of_word)

    def start_learning(self, num_of_word: int=10, num_of_letter_in_word: int=3):
        """
        Now we will start learning your new dictionary. 
        :param: num_of_word: int=10 --> number of words to be translated in each of the 2 tasks
        :param: num_of_letter_in_word: int=3 --> number of hint letter in word
        """
        self.create_learning_list()
        self.load_your_vocabulary_list()
        self.rules_learn()
        print("Let's get started")
        score1 = self.translation_learn_task(num_of_word, num_of_letter_in_word, 1)
        score2 = self.translation_learn_task(num_of_word, num_of_letter_in_word, 2)
        print('')
        if((score1+score2)/2 > 70):
            print('Good job! Keep working and you will become a champion!')
            print(f"Your total score: {(score1+score2)/2}%")
        elif((score1+score2)/2 > 30):
            print('You still need to work a little more!')
            print(f"Your total score: {(score1+score2)/2}%")
        else:
            print('Patience you must have my young padawan, you still have a lot of work to do.')
            print(f"Your total score: {(score1+score2)/2}%")

    def start_testing(self, num_of_word: int=5) -> None:
        """
        Now we will start testing your knowledge. 
        :param: num_of_word: int=5 --> number of words to be translated in each of the 2 tasks
        """
        self.create_learning_list()
        self.load_your_vocabulary_list()
        self.rules_test()
        print("Let's get started")
        score1 = self.translation_test_task(num_of_word, 1)
        score2 = self.translation_test_task(num_of_word, 2)
        print('')
        if((score1+score2)/2 > 70):
            print('Good job! Keep working and you will become a champion!')
            print(f"Your total score: {(score1+score2)/2}%")
        elif((score1+score2)/2 > 30):
            print('You still need to work a little more!')
            print(f"Your total score: {(score1+score2)/2}%")
        else:
            print('Patience you must have my young padawan, you still have a lot of work to do.')
            print(f"Your total score: {(score1+score2)/2}%")

In [9]:
lv = LearnVocabulary(path2txt='vocabulary\wladaj_i_gadaj_adv_1.txt', alias='MM')
lv.present_dictionary()

##### All words in dictionary #####
#### English -- Polish
## 1. annual -- corocznie
## 2. silent -- niemy
## 3. fetch -- przynieść
## 4. castaway -- rozbitek
## 5. discover -- odkryć
## 6. worship -- czcić


In [10]:
lv.start_learning(3, 3)

 -- Your vocabulary list (English) does exist!
 -- Your vocabulary list (Polish) does exist!
!PRINCIPLES OF LEARNING!
        You will get hints for each word in the form of additional letters.
        And then you check if you remember the word well. 
Let's get started
Exercise 1: Translate words from English into Polish.
You already know everything!
Exercise 2: Translate words from Polish into English.
You already know everything!

Good job! Keep working and you will become a champion!
Your total score: 100.0%


In [11]:
lv.start_testing(2)

 -- Your vocabulary list (English) does exist!
 -- Your vocabulary list (Polish) does exist!
!RULES TEST!
        You will get n words to translate in both languages. 
        Your progress list will be saved after each task. 
        If you translate the word correctly, it will be removed word from your progress list. 
        If you translate a word incorrectly, the word will be added to your progress list.
Let's get started
Exercise 1: Translate words from English into Polish.
You already know everything!
Exercise 2: Translate words from Polish into English.
You already know everything!

Good job! Keep working and you will become a champion!
Your total score: 100.0%
